In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy
import time
import json

In [31]:
#setting
headers = {
    'authority':'www.target.com',
    'method':'GET',
    'scheme':'https',
    'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding':'gzip, deflate, br',
    'accept-language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control':'max-age=0',
    'cookie':'visitorId=0176264E72AB0201BD56901C9116F55E; sapphire=1; UserLocation=00320|24.960|121.220|TAO|TW; criteo={%22criteo%22:%22TLivaGLROf0ylwycwLYrHSlnBdhCceME%22}; __gads=ID=36d6592eeaca8cd0:T=1606960447:S=ALNI_MZX65a77QOQAagPiNoFGWbP40PhOg; ci_pixmgr=other; _gcl_au=1.1.759698053.1606960453; kampyle_userid=deaa-e552-5eb5-1db5-f4c4-e52b-efd7-fbc3; cd_user_id=176264e960d39d-0065640583b238-c791e37-e1000-176264e960e2ae; crl8.fpcuid=a27621ad-4eff-4af1-936c-8f42cd03a90e; TealeafAkaSid=CCAtJgcSF3n3yP9gyeXUcKoR1mKdah9c; GuestLocation=328 41|24.970|121.150|TAO|TW; adaptiveSessionId=A9898352313; tlThirdPartyIds=%7B%22pt%22%3A%22v2%3A6b1a0ce68c390000a28f770ba379f3ff9a8fce9f676d470f078b417e51b22896%7C2399a78d62a6724680c5eea096607d0824dd4784d0d16a978e4e2a3694c49b5c%22%2C%22adHubTS%22%3A%22Mon%20Dec%2021%202020%2009%3A58%3A14%20GMT%2B0800%20(%E5%8F%B0%E5%8C%97%E6%A8%99%E6%BA%96%E6%99%82%E9%96%93)%22%7D; DwaGuestLocation=328 41|guanyin|TAO|TW; sessionId=15023eebbf2d611608515917046; ci_ref=null; kampyleUserSession=1608516100021; kampyleUserSessionsCount=12; kampyleSessionPageCounter=1; _uetsid=fc802280432f11eba9efe74f2bdd3d93; _uetvid=7112d4a0350a11eb92d96d0a504d82d1; ffsession={%22sessionHash%22:%221695d84ce56dc11608515892533%22%2C%22sessionHit%22:80%2C%22prevPageType%22:%22search:%20search%20results%22%2C%22prevPageName%22:%22search:%20search%20results%22%2C%22prevPageUrl%22:%22https://www.target.com/s?searchTerm=MAINSTREAM+WHITE+BREAD%22%2C%22prevSearchTerm%22:%22MAINSTREAM%20WHITE%20BREAD%22%2C%22lnkClickContent%22:null%2C%22lnkClickList%22:null%2C%22lnkClickNav%22:null%2C%22lnkPromotions%22:null%2C%22lnkClickRecommendation%22:null}',
    'sec-fetch-dest':'document',
    'sec-fetch-mode':'navigate',
    'sec-fetch-site':'same-origin',
    'sec-fetch-user':'?1',
    'upgrade-insecure-requests':'1',
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

with open('./items.txt', 'r', encoding='utf-8') as f:
    items = f.readlines()
    

for item in items:
    item = item.replace(' ','+')
    print(item,end='')

FLUID+MILK+WHITE+ONLY
YOGURT+NOT+MULTI-PACKS
SOFT+DRINKS+CAN
SOFT+DRINK+INCL
CANDY+BARS
SHREDDED+CHEESE
MAINSTREAM+WHITE+BREAD
CONDENSED+SOUP
ECONOMY+ENTREES+DINNERS
BANANAS
SOFT+DRINK+POWDER+POUCHES
POTATO+CHIPS
PASTA+CANNED
ICE+CREAM
RAMEN+NOODLES/RAMEN+CUPS
BEER
100%+PURE+JUICE
PREMIUM+ENTREES
VEGETABLES+-PLAIN
TUNA

In [33]:
for n,item in enumerate(items):
    item = item.replace(' ','+')
    #Request product data
    data_url = 'https://redsky.target.com/redsky_aggregations/v1/web/plp_search_v1?key=1&channel=WEB&count=24&default_purchasability_filter=false&include_sponsored=true&keyword=' + item + '&offset=0&page=%2Fs%2F' + item + '&platform=desktop&pricing_store_id=3991&useragent=Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F87.0.4280.88+Safari%2F537.36&visitor_id=1'
    data_res = requests.get(url=data_url,headers=headers)
    data = json.loads(data_res.text)
    skuid_mix = []
    api_mix = []

    #Find skuid
    for skuid_data in data['data']['search']['products']:
        skuid_mix.append(skuid_data['tcin'])
    
    #Find Api_key
    for api_data in data['data']['search']['products']:
        api_url = api_data['item']['enrichment']['buy_url']
        api_url = api_url + '#lnk=sametab'
        api_res = requests.get(url=api_url,headers=headers)
        api_soup = BeautifulSoup(api_res.text, 'html.parser')
        apiss=api_soup.select('script')[5]
        for apis in apiss:
            apis = apis.split('"nova":')[1].split('{"apiKey":"')[1].split('","base":')[0]
            api_mix.append(apis)  
    #Reviews
    review = ''

    for api,skuid in list(zip(api_mix,skuid_mix)):
        reviews = 'https://r2d2.target.com/ggc/v2/summary?reviewType=PRODUCT&key=' + api + '&reviewedId=' + str(skuid) + '&sortBy=most_recent&page=0&size=100&includes=reviews%2CreviewsWithPhotos'
        res_rev = requests.get(url=reviews, headers=headers)
        rev_json = json.loads(res_rev.text)
        for review_data in rev_json['reviews']['results']:
            review += review_data['text']
            review += '\n--------------------\n'
    
    item = item.replace('+','_').strip('\n')
    with open('./comment/Target_reviews/%s.txt' % (item), 'w', encoding='utf-8') as w:
            w.write(review)

    print('%d finish' % (n))
    time.sleep(2)

0 finish


KeyboardInterrupt: 